In [21]:
from keras.models import Sequential
import keras 
from keras.layers import LSTM, Dense, Dropout
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import pandas as pd 

In [2]:
import yfinance as yf
import time
from datetime import datetime

In [3]:
start_date = datetime(2023, 10, 1).strftime('%Y-%m-%d')
end_date = datetime(2024, 10, 1).strftime('%Y-%m-%d')

In [4]:
"ZEC-USD", "XMR-USD", "LTC-USD", "AXS-USD"

('ZEC-USD', 'XMR-USD', 'LTC-USD', 'AXS-USD')

In [52]:
select_crypto = "AXS-USD"
df = yf.download(select_crypto, start=start_date, end=end_date)

[*********************100%***********************]  1 of 1 completed


In [53]:
df

Price,Close,High,Low,Open,Volume
Ticker,AXS-USD,AXS-USD,AXS-USD,AXS-USD,AXS-USD
Date,,,,,
2023-10-01,4.784446,4.798980,4.580248,4.586246,24317165
2023-10-02,4.580239,4.789109,4.520734,4.784734,35502891
2023-10-03,4.510132,4.626953,4.495383,4.580125,34281462
2023-10-04,4.489252,4.510108,4.399855,4.510108,19675686
2023-10-05,4.413610,4.503926,4.402169,4.489078,13564981
...,...,...,...,...,...
2024-09-26,5.329161,5.392738,5.084387,5.161442,32438118
2024-09-27,5.417719,5.460662,5.282948,5.329161,30843699


In [54]:
df['Close_smooth'] = df['Close'].rolling(window=3).mean()
df['day_of_week'] = df.index.dayofweek
df = df.dropna()
df = df[['Close_smooth', 'Volume', 'day_of_week']]

In [59]:
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(df)

def create_seq2seq_dataset(data, time_step=60, forecast_horizon=20):
    X, y = [], []
    for i in range(len(data) - time_step - forecast_horizon):
        X.append(data[i:i + time_step])
        y.append(data[i + time_step:i + time_step + forecast_horizon, 0])  
    return np.array(X), np.array(y)


time_step = 60
forecast_horizon = 30

X, y = create_seq2seq_dataset(scaled_data, time_step, forecast_horizon)
X = X.reshape(X.shape[0], X.shape[1], scaled_data.shape[1])  # (samples, time_steps, features)

    # Train/test split
train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]
model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dropout(0.4))

model.add(LSTM(50))
model.add(Dropout(0.3))

model.add(Dense(forecast_horizon))  

model.compile(optimizer='adam', loss='mean_absolute_error')
model.summary()
history = model.fit(X_train, y_train, epochs=50, batch_size=8, validation_split=0.1)

C:\Users\asus\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_13"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm_26 (LSTM)                       │ (None, 60, 50)              │          10,800 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_26 (Dropout)                 │ (None, 60, 50)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_27 (LSTM)                       │ (None, 50)                  │          20,200 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_27 (Dropout)                 │ (None, 50)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_13 (Dense)                     │ (None, 30)                  │           1,530 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 32,530 (127.07 KB)

 Trainable params: 32,530 (127.07 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 4s 31ms/step - loss: 0.3695 - val_loss: 0.1740
Epoch 2/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - loss: 0.1913 - val_loss: 0.1947
Epoch 3/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - loss: 0.1770 - val_loss: 0.1737
Epoch 4/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: 0.1661 - val_loss: 0.1684
Epoch 5/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - loss: 0.1652 - val_loss: 0.1832
Epoch 6/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - loss: 0.1472 - val_loss: 0.1973
Epoch 7/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - loss: 0.1522 - val_loss: 0.1059
Epoch 8/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - loss: 0.1413 - val_loss: 0.1466
Epoch 9/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - loss: 0.1463 - val_loss: 0.1468
Epoch 10/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step - loss: 0.1267 - val_loss: 0.1620
Epoch 11/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - loss: 0.1264 - val_loss: 0.1838
Epoch 12/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - loss: 0.1

In [60]:
saved_model_path =  select_crypto +'_'+str(forecast_horizon)+ '.keras'
model.save(saved_model_path)